In [1]:
import pandas as pd 

df = pd.read_csv("data/data_6_3_1.csv")
print(df['항암제'].values.tolist())

FileNotFoundError: [Errno 2] No such file or directory: 'data/data_6_3_1.csv'

### 문제 1. 항암제 투여 환자들 중 '무증상' 비율을 0과 1사이로 구하시오. 

In [ ]:
df['항암제'].value_counts(normalize=True)

In [ ]:
result = df['항암제'].value_counts(normalize=True)[0]
print(result)

In [ ]:
cnt = df['항암제'] == '무증상'
total = len(df['항암제'])
result = sum(cnt) / total
print(result)

### 문제 2. 검정통계량
- 감기약의 부작용 비율과 항암제의 관찰 값이 통계적으로 유의미하게 차이가 있는지 카이제곱 검정을 이용하여 검정통계량을 구하시오. (적합도 검정)
- 가설검정
    + 귀무가설 : 감기약의 부작용과 항암제의 부작용은 동일하다.
    + 대립가설 : 감기약의 부작용과 항암제의 부작용은 다르다. 

In [26]:
df['항암제'].value_counts()

항암제
무증상     11
조금아픔     4
아픔       3
속쓰림      2
Name: count, dtype: int64

In [29]:
total = len(df['항암제'])
ratios = [0.7, 0.1, 0.05, 0.15] # 무증상, 조금아픔, 아픔, 속쓰림
val1 = df['항암제'].value_counts().to_list() # 무증상, 조금아픔, 아픔, 속쓰림
val2 = [ratios[0] * total, ratios[1] * total, ratios[2] * total, ratios[3] * total]
print(val1, val2) # 두 비율 비교

[11, 4, 3, 2] [14.0, 2.0, 1.0, 3.0]


In [30]:
import scipy.stats as stats
statistic, pvalue = stats.chisquare(val1, val2)
print(statistic)

6.976190476190475


### 문제 3. P-value 구하기
- p-value 값 구하기

In [32]:
print(pvalue)

0.07266054733847582


## 유형 2
- 주어진 데이터를 이용하여 다중선형회귀 문제를 해결하시오. 

In [2]:
import pandas as pd 

df = pd.read_csv("data/data_6_3_2.csv")
df.head()

,Ozone,Solar,Wind,Temp,Month,Day
0,41.0,190.0,7.4,67,5,1
1,36.0,118.0,8.0,72,5,2
2,12.0,149.0,12.6,74,5,3
3,18.0,313.0,11.5,62,5,4
4,NaN,NaN,14.3,56,5,5


### 문제 1. 
- 선형회귀 모델을 만들어 독립변수 Ozone의 회귀계수(Regression coefficient)를 구하시오.
    + 독립변수 : Solar(태양 에너지), Wind(바람의 세기), Ozone(오존 농도)
    + 종속변수 : Temp(온도)

In [9]:
from statsmodels.formula.api import ols
import statsmodels.formula.api as smf

model=smf.ols(formula='Temp~Solar+Wind+Ozone',data=df).fit()
coef=model.params
coef

Intercept    72.418579
Solar         0.007276
Wind         -0.322945
Ozone         0.171966
dtype: float64

In [10]:
from statsmodels.formula.api import ols

model=smf.ols(formula='Temp~Solar+Wind+Ozone',data=df).fit()
coef=model.params
coef

# 코드

Intercept    72.418579
Solar         0.007276
Wind         -0.322945
Ozone         0.171966
dtype: float64

### 문제 2. 
- Solar와 Ozone은 고정, Wind가 증가함에 따라 Temp가 감소하는지 검증하기 위한 다중선형회귀 분석을 수행하여 Wind의 회귀계수에 대한 p-value를 구하시오. (유의수준 0.05)

In [28]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   Temp   R-squared:                       0.500
Model:                            OLS   Adj. R-squared:                  0.486
Method:                 Least Squares   F-statistic:                     35.65
Date:                Fri, 12 Jan 2024   Prob (F-statistic):           4.73e-16
Time:                        13:32:47   Log-Likelihood:                -368.79
No. Observations:                 111   AIC:                             745.6
Df Residuals:                     107   BIC:                             756.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     72.4186      3.216     22.522      0.000      66.044      78.793
Solar          0.0073      0.008      0.948      0.345      -0.008       0.022
Wind          -0.3229      0.233     -1.384      0.169      -0.785       0.139
Ozone          0.1720      0.026      6.516      0.000       0.120       0.224
==============================================================================
Omnibus:                        6.409   Durbin-Watson:                   1.170
Prob(Omnibus):                  0.041   Jarque-Bera (JB):                6.615
Skew:                          -0.591   Prob(JB):                       0.0366
Kurtosis:                       2.817   Cond. No.                     1.05e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.05e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [15]:
# 코드 
model.summary()
wind_p=model.summary().tables[1].data[3][4]
wind_p

' 0.169'

### 문제 3. 
- Solar가 150, Wind가 7.5, Ozone이 45일 경우, 예측 값과 그에 대한 95% 신뢰구간을 구하시오. 

In [31]:
new=pd.DataFrame({
    'Solar':[150],
    'Wind':[7.5],
    'Ozone':[45]
})

exp=model.predict(new)
conf=model.get_prediction(new).conf_int(alpha=0.05)
print(exp,conf)

0    78.826312
dtype: float64 [[77.11261369 80.54001092]]


In [32]:
conf1= model.get_prediction(new)
summary=conf1.summary_frame(alpha=0.05)
summary

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
0,78.826312,0.864464,77.112614,80.540011,65.171661,92.480964
